In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from skimage.color import rgb2gray
from skimage.filters import gaussian
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define paths to your dataset
train_data_dir = 'D:/DATASET/CNN/ballooning/train'
batch_size = 32

In [2]:
# Data loading and preprocessing
train_datagen = ImageDataGenerator(rescale=1./255,
                                   preprocessing_function=lambda x: gaussian(rgb2gray(x), sigma=1))

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True)

Found 7973 images belonging to 2 classes.


In [3]:
# Calculate class weights for balancing
class_weights = class_weight.compute_class_weight(
    'balanced',
    np.unique(train_generator.classes),
    train_generator.classes)

NameError: name 'class_weight' is not defined

In [ ]:
# Define oversampling and downsampling ratios
oversampling_ratio = 6700 / 1273  # Class 1 / Class 0
downsampling_ratio = 1 / oversampling_ratio

# Balance the dataset
train_generator = shuffle(train_generator)  # Shuffle to ensure randomness
class_0_count = np.sum(train_generator.classes == 0)
class_1_count = np.sum(train_generator.classes == 1)

In [ ]:
import os
import shutil

# Define paths for balanced dataset
balanced_train_dir = 'D:/DATASET/CNN/ballooning/balanced_train'
class_0_dir = os.path.join(balanced_train_dir, '0')
class_1_dir = os.path.join(balanced_train_dir, '1')

# Create directories if they don't exist
os.makedirs(class_0_dir, exist_ok=True)
os.makedirs(class_1_dir, exist_ok=True)

# Move images to corresponding directories
for i, (image, label) in enumerate(train_generator):
    if label == 0:
        shutil.copy(train_generator.filepaths[i], os.path.join(class_0_dir, f'image_{i}.jpg'))
    else:
        shutil.copy(train_generator.filepaths[i], os.path.join(class_1_dir, f'image_{i}.jpg'))
    
    if i + 1 == len(train_generator):
        break

In [ ]:
if class_0_count < class_1_count:
    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True,
        subset='indices',
        seed=42,
        classes=['0'],
        interpolation='nearest')
    oversample_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(224, 224),
        batch_size=int(batch_size * oversampling_ratio),
        class_mode='binary',
        shuffle=True,
        subset='indices',
        seed=42,
        classes=['1'],
        interpolation='nearest')
    train_generator = combine_generators(train_generator, oversample_generator, class_0_count, class_1_count)
else:
    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True,
        subset='indices',
        seed=42,
        classes=['1'],
        interpolation='nearest')
    downsample_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(224, 224),
        batch_size=int(batch_size * downsampling_ratio),
        class_mode='binary',
        shuffle=True,
        subset='indices',
        seed=42,
        classes=['0'],
        interpolation='nearest')
    train_generator = combine_generators(downsample_generator, train_generator, class_0_count, class_1_count)

In [ ]:
# Define the base model
base_model = ResNet50(weights='imagenet', include_top=False)

In [ ]:
# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# Add a fully connected layer
x = Dense(1024, activation='relu')(x)
# Add a logistic layer
predictions = Dense(1, activation='sigmoid')(x)

In [ ]:
# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=10,
                    class_weight=class_weights)

In [ ]:
# Plot training accuracy and loss
plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])
plt.title('Model Training')
plt.xlabel('Epoch')
plt.legend(['Accuracy', 'Loss'], loc='upper right')
plt.show()